In [1]:
!pip install transformers
from transformers import pipeline

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 40.1 MB/s 
     |████████████████████████████████| 86 kB 5.0 MB/s 
     |████████████████████████████████| 6.6 MB 51.1 MB/s 
     |████████████████████████████████| 596 kB 54.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import spacy
import numpy as np
import seaborn as sns
from sklearn.preprocessing import StandardScaler

In [4]:
spacy.cli.download("en_core_web_lg")
nlp = spacy.load("en_core_web_lg")

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [5]:
data_train = pd.read_csv('cve_train.csv')
data_test = pd.read_csv('cve_test.csv')
a = data_train.size
b = data_test.size
total = a+b

description_train = data_train['description']
description_test = data_test['description']
cvss_train = data_train['base_score']
cvss_test = data_test['base_score']


In [6]:
data = pd.concat([data_train, data_test], ignore_index=True)
desc = data['description']
cvss = data['base_score']

print(desc[0])
print(cvss.size)

A remote code exection vulnerability was identified in HPE Intelligent Management Center (IMC) PLAT earlier than version 7.3 E0506P09.
77020


In [7]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
print(desc[1])
print('Tokenized: ', tokenizer.tokenize(desc[1]))

An information disclosure vulnerability in the Android media framework (n/a). Product: Android. Versions: 7.0, 7.1.1, 7.1.2, 8.0. Android ID: A-63873837.
Tokenized:  ['an', 'information', 'disclosure', 'vulnerability', 'in', 'the', 'android', 'media', 'framework', '(', 'n', '/', 'a', ')', '.', 'product', ':', 'android', '.', 'versions', ':', '7', '.', '0', ',', '7', '.', '1', '.', '1', ',', '7', '.', '1', '.', '2', ',', '8', '.', '0', '.', 'android', 'id', ':', 'a', '-', '63', '##8', '##7', '##38', '##37', '.']


In [ ]:
print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(desc[1])))

Token IDs:  [2019, 2592, 19380, 18130, 1999, 1996, 11924, 2865, 7705, 1006, 1050, 1013, 1037, 1007, 1012, 4031, 1024, 11924, 1012, 4617, 1024, 1021, 1012, 1014, 1010, 1021, 1012, 1015, 1012, 1015, 1010, 1021, 1012, 1015, 1012, 1016, 1010, 1022, 1012, 1014, 1012, 11924, 8909, 1024, 1037, 1011, 6191, 2620, 2581, 22025, 24434, 1012]


In [8]:
#finding the length of the longest description
max_len = 0
counter = 0

desc_new = []
cvss_new = []

for description in range(len(desc)):
    if len(desc[description]) <= 512:
        desc_new.append(desc[description])
        cvss_new.append(cvss[description])

        #tokenize the text and add CLS and SEP tokens

        input_ids = tokenizer.encode(desc[description], add_special_tokens=True)

        # Update the maximum description length
        max_len = max(max_len, len(input_ids))

print(len(desc))
print(len(desc_new))
print('Max description length: ', max_len)

77020
67152
Max description length:  294


In [9]:
# Tokenize all descriptions and map the tokens to their word IDs
import torch
input_ids = []
attention_masks = []

for description in desc_new:
    encoded_dict = tokenizer.encode_plus(
                                description,
                                add_special_tokens = True,
                                max_length = 64,
                                pad_to_max_length = True,
                                return_attention_mask = True,
                                return_tensors = 'pt'    
                            )
    input_ids.append(encoded_dict['input_ids'])
    attention_masks.append(encoded_dict['attention_mask'])

#convert the lists into tensors
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(cvss_new)

print('Original: ', desc_new[1])
print('Token IDs: ', input_ids[1])

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2291: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Original:  An information disclosure vulnerability in the Android media framework (n/a). Product: Android. Versions: 7.0, 7.1.1, 7.1.2, 8.0. Android ID: A-63873837.
Token IDs:  tensor([  101,  2019,  2592, 19380, 18130,  1999,  1996, 11924,  2865,  7705,
         1006,  1050,  1013,  1037,  1007,  1012,  4031,  1024, 11924,  1012,
         4617,  1024,  1021,  1012,  1014,  1010,  1021,  1012,  1015,  1012,
         1015,  1010,  1021,  1012,  1015,  1012,  1016,  1010,  1022,  1012,
         1014,  1012, 11924,  8909,  1024,  1037,  1011,  6191,  2620,  2581,
        22025, 24434,  1012,   102,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0])


In [10]:
from torch.utils.data import TensorDataset, random_split

# Combine the training inputs into a TensorDataset.
dataset = TensorDataset(input_ids, attention_masks, torch.Tensor(cvss_new))

# Create a 80-20 train-validation split.

# Calculate the number of samples to include in each set.
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size

# Divide the dataset by randomly selecting samples.
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))

53,721 training samples
13,431 validation samples


In [11]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# The DataLoader needs to know our batch size for training, so we specify it 
# here. For fine-tuning BERT on a specific task, the authors recommend a batch 
# size of 16 or 32.
batch_size = 32

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order. 
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
validation_dataloader = DataLoader(
            val_dataset, # The validation samples.
            sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

In [12]:
import torch.nn as nn
from transformers import BertModel
class BertRegressor(nn.Module):
    
    def __init__(self, drop_rate=0.2, freeze_bert=False):
        
        super(BertRegressor, self).__init__()
        D_in, D_out = 768, 1
        
        self.bert = \
                   BertModel.from_pretrained('bert-base-uncased')
        self.regressor = nn.Sequential(
            nn.Dropout(drop_rate),
            nn.Linear(D_in, D_out))
    def forward(self, input_ids, attention_masks):
        
        outputs = self.bert(input_ids, attention_masks)
        class_label_output = outputs[1]
        outputs = self.regressor(class_label_output)
        return outputs
model = BertRegressor(drop_rate=0.2)

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [13]:
import torch
if torch.cuda.is_available():       
    device = torch.device("cuda")
    print("Using GPU.")
else:
    print("No GPU available, using the CPU instead.")
    device = torch.device("cpu")
model.to(device)

Using GPU.


BertRegressor(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True

In [14]:
from transformers import AdamW
optimizer = AdamW(model.parameters(),
                  lr=5e-5,
                  eps=1e-8)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [15]:
from transformers import get_linear_schedule_with_warmup
epochs = 3
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer,       
                 num_warmup_steps=0, num_training_steps=total_steps)

In [16]:
loss_function = nn.MSELoss()

In [17]:
from torch.nn.utils.clip_grad import clip_grad_norm
def train(model, optimizer, scheduler, loss_function, epochs,       
          train_dataloader, device, clip_value=2):
    for epoch in range(epochs):
        print(epoch)
        print("-----")
        best_loss = 1e10
        model.train()
        for step, batch in enumerate(train_dataloader): 
            print(step)  
            batch_inputs, batch_masks, batch_labels = \
                               tuple(b.to(device) for b in batch)
            model.zero_grad()
            outputs = model(batch_inputs, batch_masks)           
            loss = loss_function(outputs.squeeze(), 
                             batch_labels.squeeze())
            loss.backward()
            clip_grad_norm(model.parameters(), clip_value)
            optimizer.step()
            scheduler.step()
                
    return model
model = train(model, optimizer, scheduler, loss_function, epochs, 
              train_dataloader, device, clip_value=2)

0
-----
0


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.


Streaming output truncated to the last 5000 lines.
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292


In [18]:
def evaluate(model, loss_function, test_dataloader, device):
    model.eval()
    test_loss, test_r2 = [], []
    for batch in test_dataloader:
        batch_inputs, batch_masks, batch_labels = \
                                 tuple(b.to(device) for b in batch)
        with torch.no_grad():
            outputs = model(batch_inputs, batch_masks)
        loss = loss_function(outputs, batch_labels)
        test_loss.append(loss.item())
        r2 = r2_score(outputs, batch_labels)
        test_r2.append(r2.item())
    return test_loss, test_r2
def r2_score(outputs, labels):
    labels_mean = torch.mean(labels)
    ss_tot = torch.sum((labels - labels_mean) ** 2)
    ss_res = torch.sum((labels - outputs) ** 2)
    r2 = 1 - ss_res / ss_tot
    return r2

In [19]:
def predict(model, dataloader, device):
    model.eval()
    output = []
    for batch in dataloader:
        batch_inputs, batch_masks, _ = \
                                  tuple(b.to(device) for b in batch)
        with torch.no_grad():
            output += model(batch_inputs, 
                            batch_masks).view(1,-1).tolist()[0]
    return output

In [20]:
y_pred = predict(model, validation_dataloader, device)

In [50]:
y_test = []
for input, target, s in validation_dataloader.dataset:
    y_test.append(s.item())

print(y_test[0])

7.5


In [54]:
from sklearn import metrics
from sklearn.metrics import r2_score


print("Mean squared error: %.3f" %metrics.mean_squared_error(y_test, y_pred))
print("Explained variance score (1 is the best): %.3f" %metrics.explained_variance_score(y_test, y_pred))
print("Max error: %.3f" %metrics.max_error(y_test, y_pred))
print("Mean absolute percentage error: %.3f" %metrics.mean_absolute_percentage_error(y_test, y_pred))

r_squared = r2_score(y_test, y_pred)
print("R2: ", r_squared)
mdape = ((pd.Series(y_test) - pd.Series(y_pred))\
         / pd.Series(y_test)).abs().median()

print(mdape)


Mean squared error: 1.186
Explained variance score (1 is the best): 0.563
Max error: 6.953
Mean absolute percentage error: 0.116
R2:  0.5606290538188612
0.06384168084776823


In [56]:
for i in range(0, 200):
    print(y_pred[i])
    print(y_test[i])
    print()

7.678284168243408
7.5

9.739988327026367
9.800000190734863

7.660674095153809
7.800000190734863

8.333394050598145
5.400000095367432

7.716854095458984
7.5

9.794928550720215
9.800000190734863

8.874789237976074
8.800000190734863

6.005958557128906
6.099999904632568

7.208150863647461
7.5

7.525514602661133
5.5

7.866910457611084
7.800000190734863

7.765662670135498
7.800000190734863

9.4248685836792
7.0

9.611680030822754
8.800000190734863

5.74800968170166
7.5

5.414416313171387
5.5

8.491324424743652
9.800000190734863

6.309554576873779
6.099999904632568

9.729783058166504
9.800000190734863

9.705227851867676
9.800000190734863

7.104827404022217
7.5

4.9805192947387695
4.300000190734863

7.2093505859375
7.0

6.581906795501709
6.800000190734863

4.8927178382873535
4.800000190734863

8.724614143371582
8.600000381469727

9.107171058654785
9.800000190734863

7.7855024337768555
7.5

5.980217933654785
5.5

4.841043472290039
5.300000190734863

9.251275062561035
8.800000190734863

5.0000357